In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess

In [6]:
EPISODE_COUNT = 3
STEPS_PER_EP = 100

SCRIPT_RUNNING = False

In [3]:
port = 5555
simTime = 10 # seconds
stepTime = 0.01  # seconds
seed = 0
simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', '../../waf --run linear-mesh'])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

Observation space shape: Box(5,)
Action space shape: Box(5,)


In [7]:
experiment = Experiment(api_key="OZwyhJHyqzPZgHEpDFL1zxhyI",
                        project_name="rl-in-wifi", workspace="wwydmanski")

experiment.log_parameter("Episode count", EPISODE_COUNT)
experiment.log_parameter("Steps per episode", STEPS_PER_EP)
 
with tqdm.trange(EPISODE_COUNT) as t:
    for i in t:
        if not SCRIPT_RUNNING:
            subprocess.Popen(['bash', '-c', '../../waf --run linear-mesh'])
            SCRIPT_RUNNING = True
        
        cumulative_reward = 0
        reward = 0
        
        obs = env.reset()
        for step in range(STEPS_PER_EP):
            action = env.action_space.sample()
            action = action * 1 + 1
            obs, reward, done, info = env.step(action)
            cumulative_reward += reward
            experiment.log_metric("Per-ep reward", cumulative_reward, step=i*STEPS_PER_EP+step)
            
            t.set_postfix(step=step, cumul_rew=cumulative_reward)
            
        experiment.log_metric("Cumulative reward", cumulative_reward, step=i)
        SCRIPT_RUNNING = False
            
experiment.end()

COMET INFO: old comet version (1.0.44) detected. current: 1.0.45 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/bdb02e212ef64b92a47b4da887eb5292

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


 33%|████████████████████                                        | 1/3 [00:06<00:12,  6.24s/it, cumul_rew=335, step=99]

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


 67%|████████████████████████████████████████                    | 2/3 [00:12<00:06,  6.11s/it, cumul_rew=376, step=99]

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|████████████████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.84s/it, cumul_rew=354, step=99]
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:             Cumulative reward: 354.0
COMET INFO:                 Per-ep reward: 354.0
COMET INFO:         sys.gpu.0.free_memory: 9732943872
COMET INFO:     sys.gpu.0.gpu_utilization: 0
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 2078216192
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
